# 컴피티션 링크
- https://www.kaggle.com/t/2e45abe9f1434b59a3358365432a48bb

# 고객별 백화점 구매기록 데이터셋
- 미혼(1) or 기혼(0) 고객을 예측
- 학습용 구매기록 데이터
    - https://drive.google.com/file/d/1tg41qXu02FK55bFa8P1Gx2URhwPUCoQQ/view?usp=sharing
- 학습용 정답 데이터
    - https://drive.google.com/file/d/1-9AcU9nAoO4SzSmqdCOYTvWNpA5Pdt3q/view?usp=sharing
- 테스트용 구매기록 데이터
    - https://drive.google.com/file/d/1-AwDfGlHm9rNtpnHIWOK96jBJYy3f2SZ/view?usp=sharing
- 제출 양식 데이터
    - https://drive.google.com/file/d/1-Qv7SlsY5Eu3bRR7Z0IUJyLkOO1Fl3y6/view?usp=sharing

In [81]:
from google.colab import drive
drive.mount('/content/drive')

- 데이터 경로 변수

In [82]:
DATA_PATH = "/content/drive/MyDrive/멋쟁이사자처럼/data/"
DATA_PATH

'/content/drive/MyDrive/멋쟁이사자처럼/data/'

- 데이터 불러오기

In [83]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [84]:
from IPython.display import display, HTML
br_html = HTML("<br><br>")
display(HTML("<h1>학습용 구매기록 데이터</h1>"), train_tr.head(), br_html)
display(HTML("<h1>학습용 정답 데이터</h1>"), train_target.head(), br_html)
display(HTML("<h1>테스트용 구매기록 데이터</h1>"), test_tr.head(), br_html)
display(HTML("<h1>제출 양식 데이터</h1>"), submit.head(), br_html)

In [85]:
display(HTML("<h1>학습용 구매기록 데이터</h1>"))
display(train_tr.info(), br_html)

display(HTML("<h1>학습용 정답 데이터</h1>"))
display(train_target.info(), br_html)

display(HTML("<h1>테스트용 구매기록 데이터</h1>"))
display(test_tr.info(), br_html)

display(HTML("<h1>제출 양식 데이터</h1>"))
display(submit.info(), br_html)

# 특성 공학(Feature Engineering)

## 날짜 형식으로 변환

In [86]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])

## 새로 만든 feature와 병합할 고객ID로만 이루어진 데이터프레임 생성

In [87]:
train_ft = train_target[["ID"]]
test_ft = submit[["ID"]]

train_ft.shape, test_ft.shape

((14940, 1), (12225, 1))

## 아동, 생식품 관련 중분류별 구매비중과 구매횟수

In [88]:
kids_lst = [
    '아동복', '아동', '완구(문화)', '문구_팬시', '완구',
    '신생아', '팬시코너(문화)', '아동잡화', '유아복', '팬시',
    '유아', '아동단품', '문구', '원목(주니어)', '원목_주니어',
    '아동특선', '문구(문화)', '유아발육기'
]

In [89]:
grocery_lst = train_tr[train_tr['대분류'] == '생식품']['중분류'].unique().tolist()

grocery_lst.extend(train_tr[train_tr['대분류'] == '생식품파트']['중분류'].unique().tolist())

grocery_lst

['정육',
 '용기보증',
 '곡물',
 '생선',
 '야채',
 '청과',
 '냉장식품',
 '기타식품',
 '음료',
 '조미료',
 '과자',
 '건식품',
 '즉석조리',
 '과자류',
 '인스턴트식품',
 '면류',
 '일용잡화',
 '일반조리',
 '건어물',
 '차류',
 '통병조림',
 '냉동식품',
 '주류',
 '인스탄트식품',
 '가정잡화',
 '건강식품',
 '상품군미지정',
 '일반식품명품',
 '야채',
 '곡물',
 '용기보증',
 '청과',
 '냉장식품',
 '음료',
 '정육',
 '과자류',
 '일용잡화',
 '생선',
 '주류',
 '건식품',
 '즉석조리',
 '조미료',
 '가정잡화',
 '면류',
 '기타식품',
 '인스탄트식품',
 '일반조리',
 '냉동식품',
 '통병조림',
 '차류',
 '건강식품']

In [90]:
# 아동 관련 중분류의 구매금액 합계
kids_semi_categories = train_tr[train_tr['중분류'].isin(kids_lst)]
for_kids = kids_semi_categories.groupby("ID")["구매가격"].sum().reset_index()
for_kids.rename(columns={"구매가격": "아동관련_구매금액"}, inplace=True)

# 아동 관련 중분류의 구매 횟수
kids_purchase_count = kids_semi_categories.groupby("ID")["구매가격"].count().reset_index()
kids_purchase_count.rename(columns={"구매가격": "아동_중분류_구매횟수"}, inplace=True)

# 전체 구매금액 합계
total = train_tr.groupby("ID")["구매가격"].sum().reset_index()
total.rename(columns={"구매가격": "전체_구매금액"}, inplace=True)

# 아동 관련 중분류 구매비중
tmp = pd.merge(for_kids, total, on="ID", how="left")
tmp["아동_중분류_구매비중"] = tmp["아동관련_구매금액"] / tmp["전체_구매금액"]

# tmp = 구매비중과 구매횟수
tmp = pd.merge(tmp, kids_purchase_count, on="ID", how="left")

train_ft = pd.merge(train_ft, tmp[["ID", "아동_중분류_구매비중", "아동_중분류_구매횟수"]], on="ID", how="left")
train_ft

,ID,아동_중분류_구매비중,아동_중분류_구매횟수
0,train_0,0.045773,2.0
1,train_1,NaN,NaN
2,train_2,NaN,NaN
3,train_3,0.004355,3.0
4,train_4,0.063928,3.0
...,...,...,...
14935,train_14935,0.097396,4.0
14936,train_14936,NaN,NaN
14937,train_14937,0.038980,1.0
14938,train_14938,NaN,NaN


In [91]:
# 아동 관련 중분류의 구매금액 합계
kids_semi_categories = test_tr[test_tr['중분류'].isin(kids_lst)]
for_kids = kids_semi_categories.groupby("ID")["구매가격"].sum().reset_index()
for_kids.rename(columns={"구매가격": "아동관련_구매금액"}, inplace=True)

# 아동 관련 중분류의 구매 횟수
kids_purchase_count = kids_semi_categories.groupby("ID")["구매가격"].count().reset_index()
kids_purchase_count.rename(columns={"구매가격": "아동_중분류_구매횟수"}, inplace=True)

# 전체 구매금액 합계
total = test_tr.groupby("ID")["구매가격"].sum().reset_index()
total.rename(columns={"구매가격": "전체_구매금액"}, inplace=True)

# 아동 관련 중분류 구매비중
tmp = pd.merge(for_kids, total, on="ID", how="left")
tmp["아동_중분류_구매비중"] = tmp["아동관련_구매금액"] / tmp["전체_구매금액"]

# tmp = 구매비중과 구매횟수
tmp = pd.merge(tmp, kids_purchase_count, on="ID", how="left")

test_ft = pd.merge(test_ft, tmp[["ID", "아동_중분류_구매비중", "아동_중분류_구매횟수"]], on="ID", how="left")
test_ft

,ID,아동_중분류_구매비중,아동_중분류_구매횟수
0,test_0,0.232786,2.0
1,test_1,NaN,NaN
2,test_2,NaN,NaN
3,test_3,NaN,NaN
4,test_4,NaN,NaN
...,...,...,...
12220,test_12220,NaN,NaN
12221,test_12221,0.388082,8.0
12222,test_12222,0.005845,1.0
12223,test_12223,0.273168,85.0


In [92]:
# 생식품 관련 중분류의 구매금액 합계
fresh_food_semi_categories = train_tr[train_tr['중분류'].isin(grocery_lst)]
for_fresh_food = fresh_food_semi_categories.groupby("ID")["구매가격"].sum().reset_index()
for_fresh_food.rename(columns={"구매가격": "생식품관련_구매금액"}, inplace=True)

# 생식품 관련 중분류의 구매횟수
purchase_count = fresh_food_semi_categories.groupby("ID")["구매가격"].count().reset_index()
purchase_count.rename(columns={"구매가격": "생식품구매횟수"}, inplace=True)

# 전체 구매금액 합계
total = train_tr.groupby("ID")["구매가격"].sum().reset_index()
total.rename(columns={"구매가격": "전체_구매금액"}, inplace=True)

# 생식품 관련 구매금액을 전체 구매금액으로 나눠 생식품 구입 비중
tmp = pd.merge(for_fresh_food, total, on="ID", how="left")
tmp["생식품_중분류_구입비중"] = tmp["생식품관련_구매금액"] / tmp["전체_구매금액"]

# tmp = 생식품 구매비중, 구매횟수
tmp = pd.merge(tmp, purchase_count, on="ID", how="left")

train_ft = pd.merge(train_ft, tmp[["ID", "생식품_중분류_구입비중", "생식품구매횟수"]], on="ID", how="left")
train_ft

,ID,아동_중분류_구매비중,아동_중분류_구매횟수,생식품_중분류_구입비중,생식품구매횟수
0,train_0,0.045773,2.0,0.421308,3.0
1,train_1,NaN,NaN,0.017038,6.0
2,train_2,NaN,NaN,0.013254,7.0
3,train_3,0.004355,3.0,0.018077,9.0
4,train_4,0.063928,3.0,0.176057,18.0
...,...,...,...,...,...
14935,train_14935,0.097396,4.0,0.015496,1.0
14936,train_14936,NaN,NaN,NaN,NaN
14937,train_14937,0.038980,1.0,0.455061,15.0
14938,train_14938,NaN,NaN,NaN,NaN


In [93]:
# 생식품 관련 중분류의 구매금액 합계
fresh_food_semi_categories = test_tr[test_tr['중분류'].isin(grocery_lst)]
for_fresh_food = fresh_food_semi_categories.groupby("ID")["구매가격"].sum().reset_index()
for_fresh_food.rename(columns={"구매가격": "생식품관련_구매금액"}, inplace=True)

# 생식품 관련 중분류의 구매횟수
purchase_count = fresh_food_semi_categories.groupby("ID")["구매가격"].count().reset_index()
purchase_count.rename(columns={"구매가격": "생식품_중분류_구매횟수"}, inplace=True)

# 전체 구매금액 합계
total = test_tr.groupby("ID")["구매가격"].sum().reset_index()
total.rename(columns={"구매가격": "전체_구매금액"}, inplace=True)

# 생식품 관련 구매금액을 전체 구매금액으로 나눠 생식품 구입 비중
tmp = pd.merge(for_fresh_food, total, on="ID", how="left")
tmp["생식품_중분류_구입비중"] = tmp["생식품관련_구매금액"] / tmp["전체_구매금액"]

# tmp = 생식품 구매비중, 구매횟수
tmp = pd.merge(tmp, purchase_count, on="ID", how="left")

test_ft = pd.merge(test_ft, tmp[["ID", "생식품_중분류_구입비중", "생식품_중분류_구매횟수"]], on="ID", how="left")
test_ft

,ID,아동_중분류_구매비중,아동_중분류_구매횟수,생식품_중분류_구입비중,생식품_중분류_구매횟수
0,test_0,0.232786,2.0,0.278363,3.0
1,test_1,NaN,NaN,NaN,NaN
2,test_2,NaN,NaN,NaN,NaN
3,test_3,NaN,NaN,NaN,NaN
4,test_4,NaN,NaN,0.022815,2.0
...,...,...,...,...,...
12220,test_12220,NaN,NaN,NaN,NaN
12221,test_12221,0.388082,8.0,NaN,NaN
12222,test_12222,0.005845,1.0,0.008620,3.0
12223,test_12223,0.273168,85.0,0.032858,18.0


# 항상 확인하기
- 학습데이터와 테스트 데이터의 피처개수는 동일해야 함

In [94]:
train_ft.shape, test_ft.shape

((14940, 5), (12225, 5))

In [96]:
train_ft.isnull().sum()

,0
ID,0
아동_중분류_구매비중,8681
아동_중분류_구매횟수,8681
생식품_중분류_구입비중,4360
생식품구매횟수,4360


In [98]:
test_ft.isnull().sum()

,0
ID,0
아동_중분류_구매비중,7136
아동_중분류_구매횟수,7136
생식품_중분류_구입비중,3420
생식품_중분류_구매횟수,3420


In [99]:
train_ft.fillna(0, inplace=True)
test_ft.fillna(0, inplace=True)

In [100]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# 추출한 피처 저장하기

In [61]:
train_ft.to_csv(f"{DATA_PATH}train_common.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_common.csv",index=False)